Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 9.1 : Data Preparation SQL

***Install pyspark***

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6e2d298b738e5f3c8c4ab6eefff73c6bdf210b87fb5e536af6db41191ba2cbe5
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

***Mount Google Drive***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh

#pwd
!ls

/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh
2.DataCopyHimani.ipynb			      Data	      SparkOutput
2_WordCountHimani.ipynb			      Data_join       SqlDataSet
3.1_WordGrepHimani.ipynb		      Data_join2      testfile1
3.2_WordMedianHimani.ipynb		      out	      testfile2
4.MR-DataJoinHimani.ipynb		      out_Datajoin    WeatherDataset
4.MR-WordCountHimani.ipynb		      outDatajoin2    wordcount_mapper.py
5.MR-DataJoin2Himani.ipynb		      out_grep	      wordcount_num0_output.txt
6.MR-WordCountReducerHimani.ipynb	      out_testfile    wordcount_num2_output.txt
7.Spark-WordCountHimani.ipynb		      out_testfile1   wordcount_reducer.py
9.Spark-Handling-missing-values-Himani.ipynb  out_testfile2
BigDataHimani.ipynb			      out_wordmedian


In [ ]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/WeatherDataset

!ls

/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/WeatherDataset
daily_weather.csv


***Load classes and daily weather data.***

In [ ]:
from pyspark.sql import SQLContext
sqlsc = SQLContext(sc)

df = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/WeatherDataset/daily_weather.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')

In [ ]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138094,3.184161180386833,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.13785928889189,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.5528134655317185,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.23801294602953,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.598209170780958,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574893,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.0787786601481,0.0,17704.0


In [ ]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [ ]:
df.count()

1095

***Remove missing values***

In [ ]:
removeAllDF = df.na.drop()

In [ ]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [ ]:
removeAllDF.count()

1064

***Impute the missing values***

In [ ]:
from pyspark.sql.functions import avg
imputeDf = df

In [ ]:
for x in imputeDf.columns:
  meanValue = removeAllDF.agg(avg(x)).first()[0]
  print(x, meanValue)
  imputeDf =  imputeDf.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


***Print imputed data summary statistics***

In [ ]:
df.describe(['air_temp_9am']).show()
imputeDf.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [ ]:
imputeDf.count()

1095